In [1]:
import cv2
import numpy as np
from tqdm import tqdm_notebook
import os

In [2]:
PATH_TO_DATA='../data_t'
PATH_TO_RESULTS='../detections'

In [3]:
try:
    os.mkdir(PATH_TO_RESULTS)
except(OSError):
    pass

In [4]:
def process_pic(img_name):
    path = os.path.join(PATH_TO_DATA, img_name)
    gray = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2GRAY)
    bgc = np.median(gray)
    low, high = 0.7 * bgc, 1.6 * bgc
    _, one = cv2.threshold(gray, low, 100, cv2.THRESH_BINARY)
    _, two = cv2.threshold(gray, high, 100, cv2.THRESH_BINARY)
    t = np.add(one, two)
    res = np.zeros(t.shape)
    res[t != 100] = 1
    nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(res.astype(np.uint8), 4, cv2.CV_32S)

    l, r = 300, 600
    selected_labels = []
    for i in range(labels.shape[0]):
        for j in range(labels.shape[1]):
            lab = labels[i][j]
            if lab > 0 and (stats[lab, 4] < l or stats[lab, 4] > r):
                labels[i][j] = 0
            elif lab > 0:
                selected_labels.append(lab)

    not_car = []
    selected_labels = set(selected_labels)

    for lab in selected_labels:
        max_side_size = np.sqrt(stats[lab, 4]) * 2.2
        if max(stats[lab, 2:4]) > max_side_size:
            not_car.append(lab)

    for nc in not_car:
        selected_labels.remove(nc)

    #imgname score xmin ymin xmax ymax
    for lab in selected_labels:
        xmin = float(stats[lab, 0])
        ymin = float(stats[lab, 1])
        xmax = float(stats[lab, 0] + stats[lab, 2])
        ymax = float(stats[lab, 1] + stats[lab, 3])

        score = 1 / np.abs(400 - stats[lab, 4])

        yield "{} {:.5} {} {} {} {}\n".format(img_name.replace('.png', ''), score, xmin, ymin, xmax, ymax)


def process_dir(dir_path=PATH_TO_DATA, filename='results_small-vehicle.txt'):
    img_names = os.listdir(dir_path)
    f = open(os.path.join(PATH_TO_RESULTS, filename), 'w+')
    for iname in tqdm_notebook(img_names):
        if not iname.endswith('.png'):
            continue
        i = 0
        for line in process_pic(iname):
            f.write(line)
            i += 1
        print('{} cars detected on img {}'.format(i, iname))

    f.close()


In [6]:
process_dir()

9 cars detected on img P0100.png
19 cars detected on img P0098.png
14 cars detected on img P0099.png
14 cars detected on img P0174.png
68 cars detected on img P0162.png
21 cars detected on img P0163.png
24 cars detected on img P0172.png
32 cars detected on img P0164.png
7 cars detected on img P0159.png
12 cars detected on img P0171.png
9 cars detected on img P0152.png
7 cars detected on img P0095.png
7 cars detected on img P0096.png



In [6]:
os.listdir()

['Folder Processor.ipynb', '.ipynb_checkpoints', 'experiment-1.ipynb']